Populate datasets with data from database (True) or from local files (False)

In [ ]:
def collectFromDatabase(db):
    
    if db == True:
        
        # Ler base de dados EDP e Atualizar a base de dados local
        this_session = Session()
        df_consumption_ = this_session.query(consumption).all()
        df_pv_ = this_session.query(PV).all()
        this_session.close()
        df_consumption_ = pd.DataFrame(df_consumption_)
        df_pv_ = pd.DataFrame(df_pv_)
        df_consumption_.to_csv("consumption.csv")
        df_pv_.to_csv("PV.csv")
        
    elif db == False:
        
        # Ler base de dados local
        df_consumption_ = pd.read_csv("consumption.csv")
        df_pv_ = pd.read_csv("PV.csv")
        df_consumption_ = df_consumption_.drop(columns=['Unnamed: 0'])
        df_pv_ = df_pv_.drop(columns=['Unnamed: 0'])
        df_consumption_.Timestamp =  pd.to_datetime(df_consumption_.Timestamp)
        df_pv_.Timestamp =  pd.to_datetime(df_pv_.Timestamp)
        
    return df_pv_, df_consumption_

Get dataset already built (True) or build new dataset (False)

In [ ]:
def getData(local):
    
    if local == True:
    
        data = pd.read_csv("dfinal.csv")
        data = data.set_index('Timestamp')
        data = data.astype('float64')
    
    else:
    
        dataset1 = np.genfromtxt('dados__Rad.dat', delimiter=',', skip_header=3, missing_values='Missing',names=True,dtype=None)
        dataset2 = np.genfromtxt('dados__Met.dat', delimiter=',', skip_header=3, missing_values='Missing',names=True,dtype=None)

        df1 = pd.DataFrame(data=dataset1)
        df2 = pd.DataFrame(data=dataset2)

        df1.columns = ["Timestamp","RECORD","Avg_GHI","Avg_DHI","Avg_POA","Avg_DNI","Avg_cosDNI","Avg_closGHI","Std_GHI","Std_DHI","Std_POA","Std_DNI","Std_cosDNI","Std_closGHI","Min_GHI","Min_DHI","Min_POA","Min_DNI","Min_cosDNI","Min_closGHI","Max_GHI","Max_DHI","Max_POA","Max_DNI","Max_cosDNI","Max_closGHI","phi","theta","SensorT"]
        df2.columns = ["Timestamp","RECORD","T_amb_min","T_amb_max","T_amb_avg","T_dp_avg","RH_min","RH_max","RH_avg","AH_min","AH_max","AH_avg","p_amb_min","p_amb_max","p_amb_avg","rho_act","v_min","v_max","v_avg","v_vectavg","v_dir_min","v_dir_max","v_dir_vectavg"]

        df1['Timestamp'] = pd.to_datetime(df1['Timestamp'].str.decode("utf-8"), format='"%Y-%m-%d %H:%M:%S"')
        df2['Timestamp'] = pd.to_datetime(df2['Timestamp'].str.decode("utf-8"), format='"%Y-%m-%d %H:%M:%S"')
        df3 = pd.merge(df1,df2, how='inner', left_index=True, right_index=True)

        PV = df_pv.copy()
        consumption = df_consumption.copy()

        consumption = consumption.set_index('Timestamp')
        df3.rename(columns={"Timestamp_x": "Timestamp", "RECORD_x": "Record"}, inplace=True)

        PV['Timestamp'] = PV['Timestamp'].dt.round('min')
        PV = PV.groupby('Timestamp').mean().reset_index()
        PV = PV.set_index('Timestamp')

        idx = pd.period_range(PV.index[0], PV.index[-1], freq='min')
        idy = pd.period_range(consumption.index[0], consumption.index[-1], freq='min')

        PV = PV.reset_index()
        PV = PV.set_index('Timestamp').resample("min").first().reset_index().reindex(columns=PV.columns)
        cols = PV.columns.difference(['I1', 'I2', 'I3', 'V1', 'V2', 'V3', 'ActPwr'])
        PV[cols] = PV[cols].ffill()
        PV['ActPwr'] = PV['ActPwr']*1000

        consumption = consumption.reset_index()
        consumption = consumption.set_index('Timestamp').resample("min").first().reset_index().reindex(columns=consumption.columns)
        cols = consumption.columns.difference(['Ir','Is','It','Vrs','Vst','Vtr','P','S'])
        consumption[cols] = consumption[cols].ffill()

        consumption = consumption.set_index('Timestamp')
        df3 = df3.set_index('Timestamp')

        PV_original = PV.copy()
        PV = PV_original[['Timestamp', 'ActPwr']].copy()

        consumption_original = consumption.copy()
        consumption = consumption_original[['P']]

        # Sun Irradiation Theoretical Calculation

        panel = solar_panel(500, 0.15, id_name='EDP')  # surface, efficiency and name
        panel.set_orientation(array([0, 0, -1]))  # upwards
        panel.set_position(38.707089, -9.148882, 0)  # LISBON latitude, longitude, altitude

        PV = PV.reset_index()
        PV['Theoretical Value'] = PV['Timestamp'].apply(lambda x: fnc(x.year, x.month, x.day, x.hour, x.minute))# year, month, day, hour, minute
        PV = PV.set_index('Timestamp')

        PV['Theoretical Value'] = PV['Theoretical Value'].shift(60, axis = 0) #passar tudo para UTC
        PV = PV.replace(np.nan, '-1')

        PREV = df4 = pd.merge(df3, PV, left_index=True, right_index=True)

        PREV = PREV.drop(columns=['Timestamp_y', 'RECORD_y'])

        PREV = PREV[['Avg_DHI', 'Avg_GHI', 'Avg_DNI', 'Avg_POA', \
                     'T_amb_avg','RH_avg', 'AH_avg', 'p_amb_avg', 'rho_act', 'v_avg', 'v_dir_vectavg',    \
                     'Theoretical Value', 'ActPwr']].copy()

        PREV = PREV.replace('-1', np.nan)

        NOITE = PREV.copy()
        NOITE['ActPwr_noite'] = NOITE.apply(f, axis=1)

        dataaux=NOITE.copy()

        INT = NOITE.copy()
        INT['Interpolation'] = INT['ActPwr_noite'].interpolate(method='polynomial', limit=30, order=2)

        dataaux['Interpolation'] = INT['Interpolation'].copy()

        INT['Interpolation'] = INT['Interpolation'].replace(np.nan, '-1000')
        INT['Interpolation_Final'] = INT.apply(s, axis=1)
        INT['Interpolation'] = INT['Interpolation'].replace('-1000',np.nan)

        INT = INT.drop(['ActPwr', 'ActPwr_noite', 'Interpolation'], axis=1)
        INT = INT.rename(columns={"Interpolation_Final": "ActPwr"})

        consumption = consumption.interpolate(method='polynomial', limit=30, order=2)

        dfinal = pd.concat([INT, consumption], axis=1, sort = False, join = 'inner')
        dfinal = dfinal.rename(columns={"Interpolation_Final": "ActPwr"})
        dfinal = dfinal.reset_index()
        dfinal = dfinal.set_index('Timestamp')

        dfinal['hour'] = dfinal.index.hour
        dfinal['day_of_month'] = dfinal.index.day
        dfinal['day_of_week'] = dfinal.index.dayofweek
        dfinal['month'] = dfinal.index.month
        dfinal['holiday'] = dfinal.apply(holiday, axis=1)
        dfinal['AvailablePower'] = 1200000 - dfinal['P'] + dfinal['ActPwr']

        dfinal = dfinal.astype('float64')

        dfinal.to_csv("dfinal.csv")
        
        data = dfinal.copy()
    
    return data

Function to compute average

In [ ]:
def Average(lst): 
    return sum(lst) / len(lst) 

Function to compute mse

In [ ]:
def mse(actual, predicted):
    
    actual = np.array(actual)
    predicted = np.array(predicted)
    
    """ Mean Squared Error """
    return np.mean(np.square(_error(actual, predicted)))

Function to compute rmse

In [ ]:
def rmse(actual, predicted):
    
    actual = np.array(actual)
    predicted = np.array(predicted)
    
    """ Root Mean Squared Error """
    return np.sqrt(mse(actual, predicted))

Function to compute mse

In [ ]:
def mae(actual, predicted):
    
    actual = np.array(actual)
    predicted = np.array(predicted)
    
    """ Mean Absolute Error """
    return np.mean(np.abs(_error(actual, predicted)))

Function to compute the solar power output based on datetime

In [ ]:
def fnc(year, month, day, hour, minute):
    panel.set_datetime(datetime(year, month, day, hour, minute))
    return panel.power()

Auxiliary function to interpolate data

In [ ]:
def s(row):
    if row['Interpolation'] == '-1000':
        return row['Theoretical Value']  
    else:
        return row['Interpolation']

Function that identifies national holidays and outputs '1' if specific day is holiday

In [ ]:
def holiday(row):
    if row['day_of_month'] == 1 and row['month'] == 1:
        return 1  
    if row['day_of_month'] == 25 and row['month'] == 2:
        return 1
    if row['day_of_month'] == 10 and row['month'] == 4:
        return 1
    if row['day_of_month'] == 12 and row['month'] == 4:
        return 1  
    if row['day_of_month'] == 25 and row['month'] == 4:
        return 1
    if row['day_of_month'] == 1 and row['month'] == 5:
        return 1  
    if row['day_of_month'] == 10 and row['month'] == 6:
        return 1  
    if row['day_of_month'] == 11 and row['month'] == 6:
        return 1
    if row['day_of_month'] == 15 and row['month'] == 8:
        return 1
    if row['day_of_month'] == 5 and row['month'] == 10:
        return 1  
    if row['day_of_month'] == 1 and row['month'] == 11:
        return 1
    if row['day_of_month'] == 1 and row['month'] == 12:
        return 1
    if row['day_of_month'] == 8 and row['month'] == 12:
        return 1  
    if row['day_of_month'] == 25 and row['month'] == 12:        
        return 1
    else:
        return 0

In [ ]:
def magnify():
    return [dict(selector="th",
                 props=[("font-size", "7pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])]

Function that neutralizes 'Avg_GHI' if it is lower than 1

In [ ]:
def f(row):
    if row['Avg_GHI'] <= 1:
        return 0
    else:
        return row['ActPwr']

Function that extracts prediction vectors with desired shape

In [ ]:
def extractPredictionVectors(real, predicted):
    
    rows_real = len(real)
    rows_predicted = len(predicted)


    real_AvailablePower_5 = []
    predicted_AvailablePower_5= []
    real_AvailablePower_10 = []
    predicted_AvailablePower_10 = []
    real_AvailablePower_15 = []
    predicted_AvailablePower_15 = []


    for i in range(rows_real):
        real_AvailablePower_5.append(real[i][0])
        real_AvailablePower_10.append(real[i][1])
        real_AvailablePower_15.append(real[i][2])


    for i in range(rows_predicted):    
        predicted_AvailablePower_5.append(predicted[i][0])
        predicted_AvailablePower_10.append(predicted[i][1])
        predicted_AvailablePower_15.append(predicted[i][2])
        
    return real_AvailablePower_5, real_AvailablePower_10, real_AvailablePower_15, predicted_AvailablePower_5, predicted_AvailablePower_10, predicted_AvailablePower_15


Function that commputes and prints training errors (RMSE, MSE, MAE)

In [ ]:
def printTrainErrors(real, predicted):

    real_AvailablePower_5, real_AvailablePower_10, real_AvailablePower_15,\
    predicted_AvailablePower_5, predicted_AvailablePower_10, predicted_AvailablePower_15 \
     = extractPredictionVectors(real, predicted)

    train_rmse_5 = rmse(real_AvailablePower_5, predicted_AvailablePower_5)
    print('Train Available Power RMSE for 5 minutes prediction: %.2f' % train_rmse_5)

    train_rmse_10 = rmse(real_AvailablePower_10, predicted_AvailablePower_10)
    print('Train Available Power RMSE for 10 minutes prediction: %.2f' % train_rmse_10)

    train_rmse_15 = rmse(real_AvailablePower_15, predicted_AvailablePower_15)
    print('Train Available Power RMSE for 15 minutes prediction: %.2f' % train_rmse_15)

    train_mse_5 = mse(real_AvailablePower_5, predicted_AvailablePower_5)
    print('Train Available Power MSE for 5 minutes prediction: %.2f' % train_mse_5)

    train_mse_10 = mse(real_AvailablePower_10, predicted_AvailablePower_10)
    print('Train Available Power MSE for 10 minutes prediction: %.2f' % train_mse_10)

    train_mse_15 = mse(real_AvailablePower_15, predicted_AvailablePower_15)
    print('Train Available Power MSE for 15 minutes prediction: %.2f' % train_mse_15)

    train_mae_5 = mae(real_AvailablePower_5, predicted_AvailablePower_5)
    print('Train Available Power MAE in 5 minutes: %.2f' % train_mae_5)

    train_mae_10 = mae(real_AvailablePower_10, predicted_AvailablePower_10)
    print('Train Available Power MAE in 10 minutes: %.2f' % train_mae_10)

    train_mae_15 = mae(real_AvailablePower_15, predicted_AvailablePower_15)
    print('Train Available Power MAE in 15 minutes: %.2f' % train_mae_15)
    
    return train_rmse_5, train_rmse_10, train_rmse_15, \
           train_mse_5, train_mse_10, train_mse_15, \
           train_mae_5, train_mae_10, train_mae_15
            

Function that commputes and prints training errors normalized (RMSE, MSE, MAE)

In [ ]:
def printTrainErrorsNormalized(real, predicted):

    real_AvailablePower_5, real_AvailablePower_10, real_AvailablePower_15,\
    predicted_AvailablePower_5, predicted_AvailablePower_10, predicted_AvailablePower_15 \
     = extractPredictionVectors(real, predicted)

    train_rmse_5 = rmse(real_AvailablePower_5, predicted_AvailablePower_5)
    print('Train Available Power RMSE for 5 minutes prediction: %.6f' % train_rmse_5)

    train_rmse_10 = rmse(real_AvailablePower_10, predicted_AvailablePower_10)
    print('Train Available Power RMSE for 10 minutes prediction: %.6f' % train_rmse_10)

    train_rmse_15 = rmse(real_AvailablePower_15, predicted_AvailablePower_15)
    print('Train Available Power RMSE for 15 minutes prediction: %.6f' % train_rmse_15)

    train_mse_5 = mse(real_AvailablePower_5, predicted_AvailablePower_5)
    print('Train Available Power MSE for 5 minutes prediction: %.6f' % train_mse_5)

    train_mse_10 = mse(real_AvailablePower_10, predicted_AvailablePower_10)
    print('Train Available Power MSE for 10 minutes prediction: %.6f' % train_mse_10)

    train_mse_15 = mse(real_AvailablePower_15, predicted_AvailablePower_15)
    print('Train Available Power MSE for 15 minutes prediction: %.6f' % train_mse_15)

    train_mae_5 = mae(real_AvailablePower_5, predicted_AvailablePower_5)
    print('Train Available Power MAE in 5 minutes: %.6f' % train_mae_5)

    train_mae_10 = mae(real_AvailablePower_10, predicted_AvailablePower_10)
    print('Train Available Power MAE in 10 minutes: %.6f' % train_mae_10)

    train_mae_15 = mae(real_AvailablePower_15, predicted_AvailablePower_15)
    print('Train Available Power MAE in 15 minutes: %.6f' % train_mae_15)
    
    return train_rmse_5, train_rmse_10, train_rmse_15, \
           train_mse_5, train_mse_10, train_mse_15, \
           train_mae_5, train_mae_10, train_mae_15
            

Function that commputes and prints validation errors (RMSE, MSE, MAE)

In [ ]:
def printValidationErrors(real, predicted):

    real_AvailablePower_5, real_AvailablePower_10, real_AvailablePower_15,\
    predicted_AvailablePower_5, predicted_AvailablePower_10, predicted_AvailablePower_15 \
     = extractPredictionVectors(real, predicted)

    validation_rmse_5 = rmse(real_AvailablePower_5, predicted_AvailablePower_5)
    print('Validation Available Power RMSE for 5 minutes prediction: %.2f' % validation_rmse_5)

    validation_rmse_10 = rmse(real_AvailablePower_10, predicted_AvailablePower_10)
    print('Validation Available Power RMSE for 10 minutes prediction: %.2f' % validation_rmse_10)

    validation_rmse_15 = rmse(real_AvailablePower_15, predicted_AvailablePower_15)
    print('Validation Available Power RMSE for 15 minutes prediction: %.2f' % validation_rmse_15)


    validation_mse_5 = mse(real_AvailablePower_5, predicted_AvailablePower_5)
    print('Validation Available Power MSE for 5 minutes prediction: %.2f' % validation_mse_5)

    validation_mse_10 = mse(real_AvailablePower_10, predicted_AvailablePower_10)
    print('Validation Available Power MSE for 10 minutes prediction: %.2f' % validation_mse_10)

    validation_mse_15 = mse(real_AvailablePower_15, predicted_AvailablePower_15)
    print('Validation Available Power MSE for 15 minutes prediction: %.2f' % validation_mse_15)


    validation_mae_5 = mae(real_AvailablePower_5, predicted_AvailablePower_5)
    print('Validation Available Power MAE in 5 minutes: %.2f' % validation_mae_5)

    validation_mae_10 = mae(real_AvailablePower_10, predicted_AvailablePower_10)
    print('Validation Available Power MAE in 10 minutes: %.2f' % validation_mae_10)

    validation_mae_15 = mae(real_AvailablePower_15, predicted_AvailablePower_15)
    print('Validation Available Power MAE in 15 minutes: %.2f' % validation_mae_15)
    
    return validation_rmse_5, validation_rmse_10, validation_rmse_15, \
           validation_mse_5, validation_mse_10, validation_mse_15, \
           validation_mae_5, validation_mae_10, validation_mae_15

Function that commputes and prints validation errors normalized (RMSE, MSE, MAE)

In [ ]:
def printValidationErrorsNormalized(real, predicted):

    real_AvailablePower_5, real_AvailablePower_10, real_AvailablePower_15,\
    predicted_AvailablePower_5, predicted_AvailablePower_10, predicted_AvailablePower_15 \
     = extractPredictionVectors(real, predicted)

    validation_rmse_5 = rmse(real_AvailablePower_5, predicted_AvailablePower_5)
    print('Validation Available Power RMSE for 5 minutes prediction: %.6f' % validation_rmse_5)

    validation_rmse_10 = rmse(real_AvailablePower_10, predicted_AvailablePower_10)
    print('Validation Available Power RMSE for 10 minutes prediction: %.6f' % validation_rmse_10)

    validation_rmse_15 = rmse(real_AvailablePower_15, predicted_AvailablePower_15)
    print('Validation Available Power RMSE for 15 minutes prediction: %.6f' % validation_rmse_15)

    validation_mse_5 = mse(real_AvailablePower_5, predicted_AvailablePower_5)
    print('Validation Available Power MSE for 5 minutes prediction: %.6f' % validation_mse_5)

    validation_mse_10 = mse(real_AvailablePower_10, predicted_AvailablePower_10)
    print('Validation Available Power MSE for 10 minutes prediction: %.6f' % validation_mse_10)

    validation_mse_15 = mse(real_AvailablePower_15, predicted_AvailablePower_15)
    print('Validation Available Power MSE for 15 minutes prediction: %.6f' % validation_mse_15)

    validation_mae_5 = mae(real_AvailablePower_5, predicted_AvailablePower_5)
    print('Validation Available Power MAE in 5 minutes: %.6f' % validation_mae_5)

    validation_mae_10 = mae(real_AvailablePower_10, predicted_AvailablePower_10)
    print('Validation Available Power MAE in 10 minutes: %.6f' % validation_mae_10)

    validation_mae_15 = mae(real_AvailablePower_15, predicted_AvailablePower_15)
    print('Validation Available Power MAE in 15 minutes: %.6f' % validation_mae_15)
    
    return validation_rmse_5, validation_rmse_10, validation_rmse_15, \
           validation_mse_5, validation_mse_10, validation_mse_15, \
           validation_mae_5, validation_mae_10, validation_mae_15

Function that commputes and prints test errors (RMSE, MSE, MAE)

In [ ]:
def printTestErrors(real, predicted):

    real_AvailablePower_5, real_AvailablePower_10, real_AvailablePower_15,\
    predicted_AvailablePower_5, predicted_AvailablePower_10, predicted_AvailablePower_15 \
     = extractPredictionVectors(real, predicted)

    test_rmse_5 = rmse(real_AvailablePower_5, predicted_AvailablePower_5)
    print('Test Available Power RMSE for 5 minutes prediction: %.2f' % test_rmse_5)

    test_rmse_10 = rmse(real_AvailablePower_10, predicted_AvailablePower_10)
    print('Test Available Power RMSE for 10 minutes prediction: %.2f' % test_rmse_10)

    test_rmse_15 = rmse(real_AvailablePower_15, predicted_AvailablePower_15)
    print('Test Available Power RMSE for 15 minutes prediction: %.2f' % test_rmse_15)

    test_mse_5 = mse(real_AvailablePower_5, predicted_AvailablePower_5)
    print('Test Available Power MSE for 5 minutes prediction: %.2f' % test_mse_5)

    test_mse_10 = mse(real_AvailablePower_10, predicted_AvailablePower_10)
    print('Test Available Power MSE for 10 minutes prediction: %.2f' % test_mse_10)

    test_mse_15 = mse(real_AvailablePower_15, predicted_AvailablePower_15)
    print('Test Available Power MSE for 15 minutes prediction: %.2f' % test_mse_15)

    test_mae_5 = mae(real_AvailablePower_5, predicted_AvailablePower_5)
    print('Test Available Power MAE in 5 minutes: %.2f' % test_mae_5)

    test_mae_10 = mae(real_AvailablePower_10, predicted_AvailablePower_10)
    print('Test Available Power MAE in 10 minutes: %.2f' % test_mae_10)

    test_mae_15 = mae(real_AvailablePower_15, predicted_AvailablePower_15)
    print('Test Available Power MAE in 15 minutes: %.2f' % test_mae_15)
    
    
    return test_rmse_5, test_rmse_10, test_rmse_15, test_mse_5, test_mse_10, test_mse_15, test_mae_5, test_mae_10, test_mae_15

Function that commputes and prints test errors normalized (RMSE, MSE, MAE)

In [ ]:
def printTestErrorsNormalized(real, predicted):

    real_AvailablePower_5, real_AvailablePower_10, real_AvailablePower_15,\
    predicted_AvailablePower_5, predicted_AvailablePower_10, predicted_AvailablePower_15 \
     = extractPredictionVectors(real, predicted)

    test_rmse_5 = rmse(real_AvailablePower_5, predicted_AvailablePower_5)
    print('Test Available Power RMSE for 5 minutes prediction: %.6f' % test_rmse_5)

    test_rmse_10 = rmse(real_AvailablePower_10, predicted_AvailablePower_10)
    print('Test Available Power RMSE for 10 minutes prediction: %.6f' % test_rmse_10)

    test_rmse_15 = rmse(real_AvailablePower_15, predicted_AvailablePower_15)
    print('Test Available Power RMSE for 15 minutes prediction: %.6f' % test_rmse_15)

    test_mse_5 = mse(real_AvailablePower_5, predicted_AvailablePower_5)
    print('Test Available Power MSE for 5 minutes prediction: %.6f' % test_mse_5)

    test_mse_10 = mse(real_AvailablePower_10, predicted_AvailablePower_10)
    print('Test Available Power MSE for 10 minutes prediction: %.6f' % test_mse_10)

    test_mse_15 = mse(real_AvailablePower_15, predicted_AvailablePower_15)
    print('Test Available Power MSE for 15 minutes prediction: %.6f' % test_mse_15)

    test_mae_5 = mae(real_AvailablePower_5, predicted_AvailablePower_5)
    print('Test Available Power MAE in 5 minutes: %.6f' % test_mae_5)

    test_mae_10 = mae(real_AvailablePower_10, predicted_AvailablePower_10)
    print('Test Available Power MAE in 10 minutes: %.6f' % test_mae_10)

    test_mae_15 = mae(real_AvailablePower_15, predicted_AvailablePower_15)
    print('Test Available Power MAE in 15 minutes: %.6f' % test_mae_15)
    
    
    return test_rmse_5, test_rmse_10, test_rmse_15, test_mse_5, test_mse_10, test_mse_15, test_mae_5, test_mae_10, test_mae_15

Function that prints graph with loss error evolution over epochs

In [ ]:
def plotLoss(history, model, dataset):
    plt.figure(figsize=(20,5))
    plt.plot(history.history['loss'],label='Train MSE')
    plt.plot(history.history['val_loss'],label='Validation MSE')
    plt.title('Model Loss')
    plt.ylabel('mse')
    plt.xlabel('epoch')
    plt.legend()
    plt.savefig('Images/' + model + '_Loss_' + dataset)
    plt.show()

Function responsible for training model, with or without validation set

In [ ]:
def train_model(model, trainX, trainY, valX, valY, batch, epoch_count, cb, val):
    
    if val == True:    
        history = model.fit(trainX, trainY, batch_size=batch, epochs=epoch_count, \
                        callbacks=cb, validation_data = (valX, valY))
    else:    
        history = model.fit(trainX, trainY, batch_size=batch, epochs=epoch_count)
    return history

Function responsible for forcing dropout while testing

In [ ]:
class PermanentDropout(Dropout):
    def __init__(self, rate, **kwargs):
        super(PermanentDropout, self).__init__(rate, **kwargs)
        self.uses_learning_phase = False

    def call(self, x, mask=None):
        if 0. < self.rate < 1.:
            noise_shape = self._get_noise_shape(x)
            x = K.dropout(x, self.rate, noise_shape)
        return x

Function responsible for creating Encoder-Decoder models

In [ ]:
def create_model(steps_before, steps_after, cnn, feature_count, units, layer, ft, intervals):
    """ 
        creates, compiles and returns a RNN model 
        @param steps_before: the number of previous time steps (input)
        @param steps_after: the number of posterior time steps (output or predictions)
        @param feature_count: the number of features in the model
        @param hidden_neurons: the number of hidden neurons per LSTM layer
    """       
    from tensorflow.keras.layers import Lambda
    from tensorflow.keras import backend as K

    
    
    if intervals == True:
    
        if layer == 'GRU':

            if cnn == True:
                model = Sequential()

                model = Sequential()
                model.add(Conv1D(filters=ft, kernel_size=2, activation='relu', input_shape=(steps_before, feature_count),padding = 'causal'))
                model.add(Conv1D(filters=ft, kernel_size=2, activation='relu'))
                model.add(MaxPooling1D(pool_size=2))
                model.add(Flatten())
                model.add(RepeatVector(steps_after))
                model.add(GRU(units, activation='relu', return_sequences=True))
                model.add(Lambda(lambda x: K.dropout(x, level=0.2)))
                model.add(TimeDistributed(Dense(1)))

            else:

                model = Sequential()
                model.add(GRU(ft, activation='relu', input_shape=(steps_before, feature_count)))
                model.add(RepeatVector(steps_after))
                model.add(GRU(units, activation='relu', return_sequences=True))
                model.add(Lambda(lambda x: K.dropout(x, level=0.2)))
                model.add(TimeDistributed(Dense(1)))
                


        elif layer == 'LSTM':

            if cnn == True:

                model = Sequential()
                model.add(Conv1D(filters=ft, kernel_size=2, activation='relu', input_shape=(steps_before, feature_count),padding = 'causal'))
                model.add(Conv1D(filters=ft, kernel_size=2, activation='relu'))
                model.add(MaxPooling1D(pool_size=2))
                model.add(Flatten())
                model.add(RepeatVector(steps_after))
                model.add(LSTM(units, activation='relu', return_sequences=True))
                model.add(Lambda(lambda x: K.dropout(x, level=0.2)))
                model.add(TimeDistributed(Dense(1)))

            else:

                model = Sequential()
                model.add(LSTM(ft, activation='relu', input_shape=(steps_before, feature_count)))
                model.add(RepeatVector(steps_after))
                model.add(LSTM(units, activation='relu', return_sequences=True))
                model.add(Lambda(lambda x: K.dropout(x, level=0.2)))
                model.add(TimeDistributed(Dense(1)))
        else:
            print('Error: Type of layer not defined')
       
    else:
    
        if layer == 'GRU':

            if cnn == True:
                model = Sequential()

                model = Sequential()
                model.add(Conv1D(filters=ft, kernel_size=2, activation='relu', input_shape=(steps_before, feature_count),padding = 'causal'))
                model.add(Conv1D(filters=ft, kernel_size=2, activation='relu'))
                model.add(MaxPooling1D(pool_size=2))
                model.add(Flatten())
                model.add(RepeatVector(steps_after))
                model.add(GRU(units, activation='relu', return_sequences=True))
                model.add(Dropout(0.2))
                model.add(TimeDistributed(Dense(1)))

            else:

                model = Sequential()
                model.add(GRU(ft, activation='relu', input_shape=(steps_before, feature_count)))
                model.add(RepeatVector(steps_after))
                model.add(GRU(units, activation='relu', return_sequences=True))
                model.add(Dropout(0.2))
                model.add(TimeDistributed(Dense(1)))

        elif layer == 'LSTM':

            if cnn == True:

                model = Sequential()
                model.add(Conv1D(filters=ft, kernel_size=2, activation='relu', input_shape=(steps_before, feature_count),padding = 'causal'))
                model.add(Conv1D(filters=ft, kernel_size=2, activation='relu'))
                model.add(MaxPooling1D(pool_size=2))
                model.add(Flatten())
                model.add(RepeatVector(steps_after))
                model.add(LSTM(units, activation='relu', return_sequences=True))
                model.add(Dropout(0.2))
                model.add(TimeDistributed(Dense(1)))

            else:

                model = Sequential()
                model.add(LSTM(ft, activation='relu', input_shape=(steps_before, feature_count)))
                model.add(RepeatVector(steps_after))
                model.add(LSTM(units, activation='relu', return_sequences=True))
                model.add(Dropout(0.2))
                model.add(TimeDistributed(Dense(1)))
        else:
            print('Error: Type of layer not defined')        
    
    model.compile(loss='mse', optimizer='adam', metrics = ['mae', 'mse', tf.keras.metrics.RootMeanSquaredError()])
    
    model.summary()
    return model

Function responsible for creating the vanilla models

In [ ]:
def create_model_vanilla(steps_before, feature_count, units, layer, intervals):
    """ 
        creates, compiles and returns a RNN model 
        @param steps_before: the number of previous time steps (input)
        @param steps_after: the number of posterior time steps (output or predictions)
        @param feature_count: the number of features in the model
        @param hidden_neurons: the number of hidden neurons per LSTM layer
    """
    
        
    from tensorflow.keras.layers import Lambda
    from tensorflow.keras import backend as K

    
    if intervals == False:
    
        if layer == 'GRU':

            model = keras.Sequential()
            model.add(GRU(units=units, input_shape=(steps_before, feature_count)))
            model.add(Dropout(0.2))
            model.add(Dense(3))

        if layer == 'LSTM':

            model = keras.Sequential()
            model.add(LSTM(units=units, input_shape=(steps_before, feature_count)))
            model.add(Dropout(0.2))
            model.add(Dense(3))

    if intervals == True: 
        
        if layer == 'GRU':

            model = keras.Sequential()
            model.add(GRU(units=units, input_shape=(steps_before, feature_count)))
            model.add(Lambda(lambda x: K.dropout(x, level=0.2)))
            model.add(Dense(3))

        if layer == 'LSTM':

            model = keras.Sequential()
            model.add(LSTM(units=units, input_shape=(steps_before, feature_count)))
            model.add(Lambda(lambda x: K.dropout(x, level=0.2)))
            model.add(Dense(3))

    
    model.compile(loss='mse', optimizer='adam', metrics = ['mae', 'mse', tf.keras.metrics.RootMeanSquaredError()])
    
    model.summary()
    return model

Function responsible for defining callback functions

In [ ]:
def callbacksFunction(model_name, dataset):
    
    path_checkpoint = './checkpoints/checkpoint_' + model_name + '_' + dataset + '.keras'
    
    callback_checkpoint = ModelCheckpoint(filepath=path_checkpoint,
                                          monitor='val_loss',
                                          verbose=1,
                                          save_weights_only=True,
                                          save_best_only=True)

    callback_early_stopping = EarlyStopping(monitor='val_loss', patience=2, verbose=1)

    callback_tensorboard = TensorBoard(log_dir='./logs/',
                                       histogram_freq=0,
                                       write_graph=False, profile_batch = 100000000)

    callback_reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                           factor=0.1,
                                           min_lr=1e-4,
                                           patience=1,
                                           verbose=1)

    class TimeHistory(keras.callbacks.Callback):
        def on_train_begin(self, logs={}):
            self.times = []

        def on_epoch_begin(self, batch, logs={}):
            self.epoch_time_start = time.time()

        def on_epoch_end(self, batch, logs={}):
            self.times.append(time.time() - self.epoch_time_start)

    time_callback = TimeHistory()

    csv_logger = CSVLogger('./training/training_' + model_name + '_' + dataset + '.log', separator=',', append=False)

    callbacks = [callback_early_stopping,
                 callback_checkpoint,
                 callback_tensorboard,
                 callback_reduce_lr,
                 time_callback,
                 csv_logger]
    
    return callbacks, path_checkpoint

Function responsible for creating random data

In [ ]:
np.random.seed(2020)
def data_gen():
    while True:
        x = np.random.rand(512, 15, 12)  # batch x time x features
        yield x, x[:, :, 0] * x[:, :, 1] < 0.25

Fucntion responsible for computing feature importance

In [ ]:

def var_importance(model):
    g = data_gen()
    x = np.concatenate([next(g)[0] for _ in range(50)]) # Get a sample of data
    orig_out = model.predict(x)
    for i in range(12):  # iterate over the three features
        new_x = x.copy()
        perturbation = np.random.normal(0.0, 0.2, size=new_x.shape[:2])
        new_x[:, :, i] = new_x[:, :, i] + perturbation
        perturbed_out = model.predict(new_x)
        effect = ((orig_out - perturbed_out) ** 2).mean() ** 0.5
        print(f'Variable {i+1}, perturbation effect: {effect:.4f}')

Functions responsible for defining variable sets

In [ ]:
def features():
    features = ['Avg_DNI',
                'Avg_DHI',
                'Avg_GHI',
                'Avg_POA',
                'RH_avg',
                'v_dir_vectavg',
                'v_avg',
                'hour', 
                'month', 
                'holiday',
                'day_of_week',
                'p_amb_avg',
                'AvailablePower']
    return features

In [ ]:
def features2():
    features = ['Avg_GHI',
                'RH_avg',
                'p_amb_avg',
                'hour', 
                'month', 
                'holiday',
                'day_of_week',
                'AvailablePower']
    return features

Function responsible for outputing reconstructed solar production signal

In [ ]:
def printReconstruction(dt):

    plt.rcParams.update({'figure.figsize':(20,10), 'figure.dpi':300})
    font = {'weight' : 'normal','size'   : 28}
    plt.rc('font', **font)
    pyplot.figure()

    dt['ActPwr_final'].plot(label='End of Phase 5', linewidth=4, color='green')
    dt['Interpolation'].plot(label='End of Phase 4', color='blue', linewidth=4)
    dt['ActPwr_noite'].plot(label='End of Phase 3',linewidth=4, color='red')

    plt.title('Production Active Power - 24/02/2020')
    plt.xlim('02/24/2020 06:30', '02/24/2020 19:00')
    plt.ylim(-5,45000 )
    plt.xlabel('Date')
    plt.ylabel('[W]')
    plt.legend()

    import matplotlib.dates as md
    ax=plt.gca()
    xfmt = md.DateFormatter('%H:%M')
    ax.xaxis.set_major_formatter(xfmt)


    axes = plt.gca()
    plt.grid()
    plt.savefig("int0")
    pyplot.show()

Function responsible for defining all datasets used

In [ ]:
def getDatasets(dfinal):
    
    a1 = dfinal[1440:102179].copy()
    v1 = dfinal[102179:122339].copy()
    a2 = dfinal[1440:122339].copy()
    v2 = dfinal[122339:142499].copy()
    a3 = dfinal[1440:142499].copy()
    v3 = dfinal[142499:162659].copy()
    a4 = dfinal[1440:162659].copy()
    v4 = dfinal[162659:182819].copy()
    a5 = dfinal[1440:182819].copy()
    t5 = dfinal[182819:202979].copy()
    d6 = dfinal[1440:202979].copy()
    
    return a1, a2, a3, a4, a5, v1, v2, v3, v4, t5, d6

Function reponsible for implementing the Naive prediction

In [ ]:
def Naive_Predict(val):
    
    n_pre = 60
    n_post = 15

    vY = []

    for i in range(len(val)):

        ar = np.array(([row[12] for row in val[i]][-1], [row[12] for row in val[i]][-1], [row[12] for row in val[i]][-1]))
        B = ar.reshape(-1, len(ar))
        vY.append(B)

    return np.array(vY)